### Test case LVV-T115: Verify implementation of Calibration Production Processing

Execute CPP on a variety of representative cadences, and verify that the calibration pipeline correctly produces necessary calibration products.

Discussion: To demonstrate all aspects of Calibration Production Processing (CPP), we execute the `ci_cpp` pipeline that is used for continuous integration (CI) of the CPP codebase. This pipeline produces all calibration products from a small input dataset, and runs `cp_verify` to verify that the output calibrations meet quality requirements.

After executing the `ci_cpp` processing, this notebook will examine the outputs.

In [1]:
import numpy as np

from lsst.daf.butler import Butler

Initialize the butler corresponding to the `ci_cpp` processing that was executed in "~/u/repos/ci_cpp_gen3/DATA".

In [2]:
butler = Butler('~/u/repos/ci_cpp_gen3/DATA/')

Examine the list of collections that were created in this repository.

In [3]:
butler.collections.query_info('*')

[CollectionInfo(name='LATISS/calib', type=<CollectionType.CALIBRATION: 4>, doc='', children=(), parents=None, dataset_types=None),
 CollectionInfo(name='LATISS/calib/unbounded', type=<CollectionType.RUN: 1>, doc='', children=(), parents=None, dataset_types=None),
 CollectionInfo(name='LATISS/calib/curated/19700101T000000Z', type=<CollectionType.RUN: 1>, doc='', children=(), parents=None, dataset_types=None),
 CollectionInfo(name='LATISS/calib/curated/20180101T000000Z', type=<CollectionType.RUN: 1>, doc='', children=(), parents=None, dataset_types=None),
 CollectionInfo(name='LATISS/calib/curated/20221005T000000Z', type=<CollectionType.RUN: 1>, doc='', children=(), parents=None, dataset_types=None),
 CollectionInfo(name='LATISS/raw/all', type=<CollectionType.RUN: 1>, doc='', children=(), parents=None, dataset_types=None),
 CollectionInfo(name='ci_cpp_biasBootstrap/run', type=<CollectionType.RUN: 1>, doc='', children=(), parents=None, dataset_types=None),
 CollectionInfo(name='ci_cpp_dar

We see that there are collections corresponding to a variety of calibration object types, including brighter-fatter kernel (bfk), photon transfer curves (ptc), biases, darks, flats, and many others. There are also validation collections (denoted with "cpv" in their names) that result from the `cp_verify` process.

#### Brighter-fatter kernel

Retrieve a `bfk` dataset and examine it to confirm that it is well-formed.

In [4]:
bfk_refs = butler.query_datasets('bfk', collections='ci_cpp_bfk')
bfk_refs[0]

DatasetRef(DatasetType('bfk', {instrument, detector}, BrighterFatterKernel, isCalibration=True), {instrument: 'LATISS', detector: 0}, run='ci_cpp_bfk/20250508T180115Z', id=aa17fe95-fd7f-4cd4-a75e-5e2790958618)

In [5]:
bfk = butler.get(bfk_refs[0])

In [6]:
bfk.toTable()[0].columns

<TableColumns names=('AMPLIFIER','EXP_ID_MASK','RAW_MEANS','RAW_VARIANCES','RAW_XCORRS','GAIN','NOISE','MEAN_XCORRS','KERNEL','VALID')>

In [7]:
print('BFK keys: ', bfk.meanXcorrs.keys())
bfk.meanXcorrs['C10'][1]

BFK keys:  dict_keys(['C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C07', 'C06', 'C05', 'C04', 'C03', 'C02', 'C01', 'C00'])


array([ 0.00000000e+00, -2.73163779e-09,  3.96425708e-09, -1.32315672e-08,
       -7.15069939e-09, -1.39779180e-08, -7.21418731e-09, -3.82108189e-09,
       -5.18853051e-08, -3.82108189e-09, -7.21418731e-09, -1.39779180e-08,
       -7.15069939e-09, -1.32315672e-08,  3.96425708e-09, -2.73163779e-09,
        0.00000000e+00], dtype='>f8')

We see that the `bfk` has expected table columns, entries for each of the 16 amps, and reasonable values for one of the amps (C10).

#### Defects mask

Retrieve a `defects` dataset and confirm that it is well formed.

In [8]:
defects_refs = butler.query_datasets('defects', collections='ci_cpp_defects')
defects_refs[0]

DatasetRef(DatasetType('defects', {instrument, detector}, Defects, isCalibration=True), {instrument: 'LATISS', detector: 0}, run='ci_cpp_defects/20250508T171844Z', id=05e4929c-b27d-43c9-aef9-f7597059c038)

In [9]:
defects = butler.get(defects_refs[0])

In [10]:
defects.toTable()

[<Table length=1796>
   x0    y0  width height
 int64 int64 int64 int64 
 ----- ----- ----- ------
     0     0  4072      5
     0     5     5   3995
   434     5     2      1
  1158     5     1      1
  1782     5     3      1
  4067     5     5   3995
   435     6     1      1
     5  1529     1      1
   605  2000     4   2000
   ...   ...   ...    ...
  3880  3967     1      1
   262  3969     1      1
  3683  3980     1      1
  1981  3987     2      3
  1983  3988     1      2
  4061  3989     1      2
  4059  3990     2      2
  4058  3991     1      3
  4057  3992     1      2
  2792  3992     1      2]

We see that the defects mask has the expected format, with "blocks" of defects defined by their initial x, y positions (x0, y0) and their width and height.

#### Photon transfer curve (PTC)

Retrieve a `ptc` dataset and confirm it is well formed.

In [11]:
ptc_refs = butler.query_datasets('ptc', collections='ci_cpp_ptc')
ptc_refs[0]

DatasetRef(DatasetType('ptc', {instrument, detector}, PhotonTransferCurveDataset, isCalibration=True), {instrument: 'LATISS', detector: 0}, run='ci_cpp_ptc/20250508T173630Z', id=c17caf12-d9e4-4306-a767-ba6136843186)

In [12]:
ptc = butler.get(ptc_refs[0])

In [13]:
print(" ptc gains: ", ptc.gain, "\n\n", "ptc turnoffs: ", ptc.ptcTurnoff, "\n\n",
      "ptc amp offsets (detector C10): \n", ptc.ampOffsets['C10'])

 ptc gains:  {'C10': 1.0229103371986488, 'C11': 1.0199883671487262, 'C12': 1.0191040394294557, 'C13': 1.0195022157826048, 'C14': 1.0104110935362776, 'C15': 1.0078748864525646, 'C16': 1.0119721241658262, 'C17': 0.9893530506651513, 'C07': 1.0438400644177985, 'C06': 1.0433286910601023, 'C05': 1.0341635986113358, 'C04': 1.0385946410916136, 'C03': 1.0408398298725232, 'C02': 1.0346223068628317, 'C01': 1.0370898980923489, 'C00': 1.0378785129533388} 

 ptc turnoffs:  {'C10': 100620.3368756783, 'C11': 90936.5462155696, 'C12': 85961.23808694436, 'C13': 116440.3616302361, 'C14': 107120.90791769812, 'C15': 123031.98099914874, 'C16': 97076.0165297454, 'C17': 105180.04773112119, 'C07': 109102.08788589927, 'C06': 98566.43955705804, 'C05': 119148.98053363284, 'C04': 98557.70596866481, 'C03': 98265.60778489435, 'C02': 103852.5195235509, 'C01': 118432.18385136726, 'C00': 118210.21971685355} 

 ptc amp offsets (detector C10): 
 [   12.13849281    23.64021393    36.74912906    49.40857021
    64.21265276 

We see that the PTC contains expected quantities for each of the 16 amps, including gain, the PTC turnoff, and amp offsets, among many other quantities.

#### Bias, dark, and flat frames

Retrieve a bias, dark and flat from the newly-created calib collection, and print their bounding box and mean/median/stdev pixel values to the screen. We will take these quantities as evidence that these are well-formed images.

In [14]:
calib_img_types = ['bias', 'dark', 'flat']

for imtype in calib_img_types:
    imrefs = butler.query_datasets(f"{imtype}", collections=f"ci_cpp_{imtype}")
    print(imrefs[0])
    img = butler.get(imrefs[0])
    print("Image bounding box: \n", img.getBBox(), "\n")
    print(f" {imtype} mean pixel value: ", np.mean(img.getImage().array), "\n",
          f"{imtype} median pixel value: ", np.median(img.getImage().array), "\n",
          f"{imtype} pixel value stdev: ", np.std(img.getImage().array))
    print("\n")

bias@{instrument: 'LATISS', detector: 0} [sc=ExposureF] (run=ci_cpp_bias/20250508T180902Z id=a10b60f1-fc6a-47fc-9cd8-9d8209af0a0d)
Image bounding box: 
 (minimum=(0, 0), maximum=(4071, 3999)) 

 bias mean pixel value:  3.2099986 
 bias median pixel value:  -0.035557225 
 bias pixel value stdev:  296.44974


dark@{instrument: 'LATISS', detector: 0} [sc=ExposureF] (run=ci_cpp_dark/20250508T181519Z id=321c21ee-a2e7-4874-9c9b-abda06ce4ee8)
Image bounding box: 
 (minimum=(0, 0), maximum=(4071, 3999)) 

 dark mean pixel value:  1.3993032 
 dark median pixel value:  0.0052579595 
 dark pixel value stdev:  144.12141


flat@{instrument: 'LATISS', detector: 0, physical_filter: 'RG610~empty', band: 'r'} [sc=ExposureF] (run=ci_cpp_flat/20250508T182205Z id=e765176b-2148-4f74-becb-f6578fc904ec)
Image bounding box: 
 (minimum=(0, 0), maximum=(4071, 3999)) 

 flat mean pixel value:  0.9993354 
 flat median pixel value:  0.9994129 
 flat pixel value stdev:  0.018624557




## Results

We have confirmed that running `ci_cpp` has produced the expected datasets, and by inspecting a few of them, have confirmed that they are well-formed datasets with the expected characteristics. The results of this test is a **Pass**.